# **NBA Winner Predictor**

Importing the necessary libraries and dataset.

Data Collection

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
import io
data=files.upload()

Saving nba_games.csv to nba_games.csv


In [ ]:
df=pd.read_csv(io.StringIO(data['nba_games.csv'].decode('utf-8')))

Exploratory Data Analysis(EDA)

Data Cleaning - Keeping the necessary columns.

In [ ]:
import pandas as pd

# Assume you have a DataFrame called 'df' containing the original dataset

# Identify the relevant columns
relevant_columns = ['pts',	'team',	'total',	'home',	'pts_opp',
                    'team_opp',	'total_opp',	'home_opp','won']

# Create a new dataset with relevant columns
new_df = df[relevant_columns].copy()

# Drop irrelevant columns from the new dataset (optional)
# new_df.drop(['column4', 'column5'], axis=1, inplace=True)

# Now 'new_df' contains only the relevant columns
new_df

,pts,team,total,home,pts_opp,team_opp,total_opp,home_opp,won
0,98.0,ATL,98,0,95.0,DAL,95,1,True
1,95.0,DAL,95,1,98.0,ATL,98,0,False
2,99.0,MIN,99,0,107.0,SAS,107,1,False
3,107.0,SAS,107,1,99.0,MIN,99,0,True
4,75.0,ORL,75,0,92.0,MEM,92,1,False
...,...,...,...,...,...,...,...,...,...
17767,108.0,LAC,108,1,92.0,OKC,92,0,True
17768,113.0,LAC,113,0,108.0,ORL,108,1,True
17769,108.0,ORL,108,1,113.0,LAC,113,0,False
17770,117.0,BOS,117,0,106.0,MIA,106,1,True


Data Cleaning - Keeping consistent teams.

In [ ]:
const_teams = ['BOS','CHI','PHI','MIL','DET','LAL','CLE','ATL']

In [ ]:
print(f'Before Removing Inconsistent Teams : {new_df.shape}')
new_df = new_df[(new_df['team'].isin(const_teams)) & (new_df['team_opp'].isin(const_teams))]
print(f'After Removing Irrelevant Columns : {new_df.shape}')
print(f"Consistent Teams : \n{new_df['team'].unique()}")
new_df.head()

Before Removing Inconsistent Teams : (17772, 9)
After Removing Irrelevant Columns : (1396, 9)
Consistent Teams : 
['DET' 'PHI' 'BOS' 'MIL' 'ATL' 'CLE' 'CHI' 'LAL']


,pts,team,total,home,pts_opp,team_opp,total_opp,home_opp,won
64,104.0,DET,104,0,118.0,PHI,118,1,False
65,118.0,PHI,118,1,104.0,DET,104,0,True
218,92.0,BOS,92,0,116.0,MIL,116,1,False
219,116.0,MIL,116,1,92.0,BOS,92,0,True
260,111.0,ATL,111,0,112.0,CLE,112,1,False


Data Preprocessing - Performing One Hot Encoding, Label Encoding and Column Transformation.

Feature Engineering

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Perform one-hot encoding on 'team' and 'team_opp' columns
onehot_cols = ['team', 'team_opp']
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = pd.DataFrame(onehot_encoder.fit_transform(new_df[onehot_cols]))

# Retrieve feature names for one-hot encoded columns
onehot_feature_names = []
for feature in onehot_encoder.categories_:
    onehot_feature_names.extend(feature)

onehot_encoded.columns = onehot_feature_names

# Perform label encoding on 'home' and 'home_opp' columns
label_cols = ['home', 'home_opp']
label_encoder = LabelEncoder()
for col in label_cols:
    new_df[col] = label_encoder.fit_transform(new_df[col])

# Select relevant columns for analysis
selected_cols = ['pts', 'total', 'home', 'pts_opp', 'total_opp', 'home_opp', 'won']
selected_final_df = pd.concat([new_df[selected_cols], onehot_encoded], axis=1)

# Perform any additional preprocessing or feature engineering as needed

# Split the data into input features (X) and target variable (y)
X = selected_final_df.drop('won', axis=1)
y = selected_final_df['won']

# Now you can use X and y in your random forest algorithm for prediction



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
<ipython-input-8-2d134e36bc8b>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[col] = label_encoder.fit_transform(new_df[col])
<ipython-input-8-2d134e36bc8b>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[col] = label_encod

Model Building

Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.20, shuffle=True)
print(f"Training Set : {train_X.shape}\nTesting Set : {test_X.shape}")

Training Set : (2168, 22)
Testing Set : (542, 22)


In [ ]:
models = dict()

Data Cleaning - Removing the missing values from the features(X) and labels(y).

In [ ]:
# Drop rows with missing values from train_X
train_X = train_X.dropna()

# Drop rows with missing values from test_X
test_X = test_X.dropna()
# Drop corresponding rows from train_y
train_y = train_y[train_X.index]

# Drop corresponding rows from test_y
test_y = test_y[test_X.index]



In [ ]:
train_X.head()

,pts,total,home,pts_opp,total_opp,home_opp,ATL,BOS,CHI,CLE,...,MIL,PHI,ATL,BOS,CHI,CLE,DET,LAL,MIL,PHI
261,112.0,112.0,1.0,111.0,111.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1278,87.0,87.0,0.0,105.0,105.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
891,99.0,99.0,1.0,95.0,95.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
371,105.0,105.0,1.0,96.0,96.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
808,114.0,114.0,0.0,106.0,106.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Predictive Modeling

Random Forest Regressor

The best model that gives higher accuracy than the rest.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer

# Create an instance of the KNNImputer with the desired number of neighbors
imputer = KNNImputer(n_neighbors=5)

# Impute missing values in the training data
imputed_train_X = imputer.fit_transform(train_X)

# Create an instance of the random forest regressor
forest = RandomForestRegressor()

# Train the model with the imputed training data
forest.fit(imputed_train_X, train_y)

train_score_forest = str(forest.score(train_X, train_y) * 100)
test_score_forest = str(forest.score(test_X, test_y) * 100)
print(f'Train Score: {train_score_forest[:5]}%\nTest Score: {test_score_forest[:5]}%')
models["forest"] = test_score_forest




Train Score: 97.01%
Test Score: 65.41%


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [ ]:
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
print("Random Forest Regression - Model Evaluation")
print("Mean Absolute Error (MAE): {}".format(mae(test_y, forest.predict(test_X))))
print("Mean Squared Error (MSE): {}".format(mse(test_y, forest.predict(test_X))))
print("Root Mean Squared Error (RMSE): {}".format(np.sqrt(mse(test_y, forest.predict(test_X)))))

Random Forest Regression - Model Evaluation
Mean Absolute Error (MAE): 0.18933333333333335
Mean Squared Error (MSE): 0.08606666666666667
Root Mean Squared Error (RMSE): 0.2933712096758417


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


Checking feature compatibility

In [ ]:
import numpy as np

# Assuming X_train is your training data
print("Number of features in training data:", np.shape(train_X))
np.shape(test_X)


Number of features in training data: (67, 22)


(15, 22)

Prediction

In [ ]:
 def score_predict(team,team_opp, pts,total,home,pts_opp,total_opp,home_opp,season, won, model=forest):
  prediction_array = []
  #Team 
  if team == 'DET':
    prediction_array = prediction_array + [1,0,0,0,0,0,0,0]
  elif team == 'PHI':
    prediction_array = prediction_array + [0,1,0,0,0,0,0,0]
  elif team == 'BOS':
    prediction_array = prediction_array + [0,0,1,0,0,0,0,0]
  elif team == 'MIL':
    prediction_array = prediction_array + [0,0,0,1,0,0,0,0]
  elif team == 'ATL':
    prediction_array = prediction_array + [0,0,0,0,1,0,0,0]
  elif team == 'CLE':
    prediction_array = prediction_array + [0,0,0,0,0,1,0,0]
  elif team == 'CHI':
    prediction_array = prediction_array + [0,0,0,0,0,0,1,0]
  elif team == 'LAL':
    prediction_array = prediction_array + [0,0,0,0,0,0,0,1]
  #Team_opp
  if team_opp == 'DET':
    prediction_array = prediction_array + [1,0,0,0,0,0,0,0]
  elif team_opp == 'PHI':
    prediction_array = prediction_array + [0,1,0,0,0,0,0,0]
  elif team_opp == 'BOS':
    prediction_array = prediction_array + [0,0,1,0,0,0,0,0]
  elif team_opp == 'MIL':
    prediction_array = prediction_array + [0,0,0,1,0,0,0,0]
  elif team_opp == 'ATL':
    prediction_array = prediction_array + [0,0,0,0,1,0,0,0]
  elif team_opp == 'CLE':
    prediction_array = prediction_array + [0,0,0,0,0,1,0,0]
  elif team_opp == 'CHI':
    prediction_array = prediction_array + [0,0,0,0,0,0,1,0]
  elif team_opp == 'LAL':
    prediction_array = prediction_array + [0,0,0,0,0,0,0,1]
  prediction_array = prediction_array + [pts,total,home,pts_opp,total_opp,home_opp,season,won]
  prediction_array = np.array([prediction_array])
  pred = model.predict(prediction_array)
  return int(round(pred[0])) 

In [ ]:
# Define the score_predict function
def score_predict(team, team_opp, pts, total, home, pts_opp, total_opp, home_opp, won, model):
    prediction_array = []
    
    # Team
    if team == 'DET':
        prediction_array = prediction_array + [1, 0, 0, 0, 0, 0, 0, 0]
    elif team == 'PHI':
        prediction_array = prediction_array + [0, 1, 0, 0, 0, 0, 0, 0]
    elif team == 'BOS':
        prediction_array = prediction_array + [0, 0, 1, 0, 0, 0, 0, 0]
    elif team == 'MIL':
        prediction_array = prediction_array + [0, 0, 0, 1, 0, 0, 0, 0]
    elif team == 'ATL':
        prediction_array = prediction_array + [0, 0, 0, 0, 1, 0, 0, 0]
    elif team == 'CLE':
        prediction_array = prediction_array + [0, 0, 0, 0, 0, 1, 0, 0]
    elif team == 'CHI':
        prediction_array = prediction_array + [0, 0, 0, 0, 0, 0, 1, 0]
    elif team == 'LAL':
        prediction_array = prediction_array + [0, 0, 0, 0, 0, 0, 0, 1]
    
    # Team_opp
    if team_opp == 'DET':
        prediction_array = prediction_array + [1, 0, 0, 0, 0, 0, 0, 0]
    elif team_opp == 'PHI':
        prediction_array = prediction_array + [0, 1, 0, 0, 0, 0, 0, 0]
    elif team_opp == 'BOS':
        prediction_array = prediction_array + [0, 0, 1, 0, 0, 0, 0, 0]
    elif team_opp == 'MIL':
        prediction_array = prediction_array + [0, 0, 0, 1, 0, 0, 0, 0]
    elif team_opp == 'ATL':
        prediction_array = prediction_array + [0, 0, 0, 0, 1, 0, 0, 0]
    elif team_opp == 'CLE':
        prediction_array = prediction_array + [0, 0, 0, 0, 0, 1, 0, 0]
    elif team_opp == 'CHI':
        prediction_array = prediction_array + [0, 0, 0, 0, 0, 0, 1, 0]
    elif team_opp == 'LAL':
        prediction_array = prediction_array + [0, 0, 0, 0, 0, 0, 0, 1]
    
    prediction_array = prediction_array + [pts, total, home, pts_opp, total_opp, home_opp]
    prediction_array = np.array([prediction_array])
    pred = model.predict(prediction_array)
    return int(round(pred[0]))


Testcases

Testcase 1

In [ ]:
team = 'BOS'
team_opp = 'CHI'
pts = 110
total = 210
home = 1
pts_opp = 100
total_opp = 200
home_opp = 0
won = 30

# Call the score_predict function
prediction = score_predict(team, team_opp, pts, total, home, pts_opp, total_opp, home_opp, won, model=forest)
print("Predicted winner:", team if prediction == 1 else team_opp)


Predicted winner: BOS


Testcase 2

In [ ]:
team = 'PHI'
team_opp = 'MIL'
pts = 115
total = 215
home = 0
pts_opp = 110
total_opp = 220
home_opp = 1
won = 25

# Call the score_predict function
prediction = score_predict(team, team_opp, pts, total, home, pts_opp, total_opp, home_opp, won, model=forest)
print("Predicted winner:", team if prediction == 1 else team_opp)


Predicted winner: PHI


Testcase 3

In [ ]:
team = 'BOS'
team_opp = 'PHI'
pts = 90
total = 180
home = 1
pts_opp = 85
total_opp = 175
home_opp = 0
won = 45

# Call the score_predict function
prediction = score_predict(team, team_opp, pts, total, home, pts_opp, total_opp, home_opp, won, model=forest)
print("Predicted winner:", team if prediction == 1 else team_opp)


Predicted winner: BOS


Testcase 4

In [ ]:
team = 'CLE'
team_opp = 'CHI'
pts = 95
total = 190
home = 1
pts_opp = 90
total_opp = 180
home_opp = 0
won = 50

# Call the score_predict function
prediction = score_predict(team, team_opp, pts, total, home, pts_opp, total_opp, home_opp, won, model=forest)
print("Predicted winner:", team if prediction == 1 else team_opp)


Predicted winner: CLE


Testcase 5

In [ ]:
team = 'CHI'
team_opp = 'ATL'
pts = 105
total = 210
home = 1
pts_opp = 100
total_opp = 200
home_opp = 0
won = 10

# Call the score_predict function
prediction = score_predict(team, team_opp, pts, total, home, pts_opp, total_opp, home_opp, won, model=forest)
print("Predicted winner:", team if prediction == 1 else team_opp)


Predicted winner: CHI


Export model

Here we load our model in a pkl(pickle) file and give the same code to .py file so that both are connected and we can then perform predictions.

In [ ]:
import pickle
filename = "ml_model.pkl"
pickle.dump(forest, open(filename, "wb"))